# Общий практикум. Задача №338
## Бесконтактное определение проводимости

**Цель работы:** ознакомление с бесконтактным методом измерения электропроводимости вещества. Метод использует дифференциальный трансформатор как фазометрическое устройство для определения частотной зависимости фазового сдвига между переменным внешним магнитным полем и намагниченностью опбразца, помещённого в это поле. Удельная электрическая проводимость материала образца может быть найдена из этой зависимости.

In [49]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['figure.dpi'] = 100

### Описание метода
При электронной эмиссии каждый электрон, покидающий катод, уносит с собой энергию, часть которой - работа выхода $A$. Если катод электронной лампы нагреть до температур термоэмиссии, то вблизи катода создаётся электронное облако. Если же теперь разомкнуть анодную цепь, то число электронов, покидающих катод, приравняется к числу электронов, возвращающихся на катод из электронного облака. При замыкании анодной цепи электроны движутся от катода к аноду, и как следствие темпераура катода падает. Это явление аналогично переносу энергии молекулами жидкости при испарении.

In [1]:
data1 = np.loadtxt('data1.txt')

print ('k = ({k} ± {dk}) МэВ'.format(k=k, dk=dk))
print ('b = ({b} ± {db}) МэВ'.format(b=b, db=db))

NameError: name 'np' is not defined

### Упражнение 2.

Оценить разрешающую способность $\gamma$-спектрометра, определив величину Γ — ширину пика на половине его высоты.

In [55]:
data2 = np.loadtxt('data2.txt')

G = np.absolute(data2*k + b)

co = G[0:2]
eu = G[2:5]
unt = G[5:]

print('Co-60 разрешение')
for i, j in enumerate(co):
  print('{i} пик: {j}'.format(i=i+1, j=j), 'МэВ')

print('\nEu-152 разрешение')
for i, j in enumerate(eu):
  print('{i} пик: {j}'.format(i=i+1, j=j), 'МэВ')

print('\nНеизвестный источник разрешение')
for i, j in enumerate(unt):
  print('{i} пик: {j}'.format(i=i+1, j=j), 'МэВ')

Co-60 разрешение
1 пик: 0.06610546228496568 МэВ
2 пик: 0.06761270290020978 МэВ

Eu-152 разрешение
1 пик: 0.0009812923055513671 МэВ
2 пик: 0.007282000453972141 МэВ
3 пик: 0.016744648733044733 МэВ

Неизвестный источник разрешение
1 пик: 0.023175963335896162 МэВ
2 пик: 0.04344208260767217 МэВ
3 пик: 0.03469039688296001 МэВ
4 пик: 0.07232822787187139 МэВ


### Упражнение 3
Определить энергию пика полного поглощения по данным ранее выполненной градуировки. По разрешающей способности спектрометра и полученным распределениям оценить ошибку определения энергии $\Delta E$ для $\gamma$-квантов от неизвестного источника. Представить результат в виде 
$$\begin{equation}E(X) = E ± ∆E\end{equation}$$

In [53]:
data3 = np.loadtxt('data3.txt')

e_unt = k*data3 + b
de_unt = np.sqrt(dk**2+db**2+unt**2)

print('Энергия пиков неизвестного источника')
for i in range(data3.shape[0]):
  print('{i} пик: ({e} ± {de}) МэВ'.format(i=i, e=e_unt[i], de=de_unt[i]))

Энергия пиков неизвестного источника
0 пик: (0.31743862483531093 ± 0.0237948370374428) МэВ
1 пик: (0.8728215067445034 ± 0.0437753758908301) МэВ
2 пик: (2.403139439864201 ± 0.035106874383643696) МэВ
3 пик: (6.035894773181361 ± 0.07252890141300232) МэВ
